# Intro

## loading required data and libs

## importing

In [2]:
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import json
import inspect
from pathlib import Path
# import pylev
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
# from sentence_transformers import SentenceTransformer, util
import torch
import tokenize
from tokenize import TokenError
import io
tqdm.pandas()

import nltk
import re
import string

import numpy as np
import pandas as pd
from scipy import sparse
import sys
import textwrap
import wandb
import spacy
ner = spacy.load("en_core_web_lg")

from sklearn.model_selection import GroupShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer

pd.options.display.width = 180
pd.options.display.max_colwidth = 120

data_dir = Path('data/')
sys.path.append(str('AI4Code'))
%load_ext autoreload
%autoreload 2

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
# CONFIG = {'competition': 'AI4Code', '_wandb_kernel': 'aot'}
# wandb.init(project='AI4Code', name='exploration', config=CONFIG)

## text utils

In [4]:
from sklearn.pipeline import Pipeline

In [5]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = text.strip()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    combined_text = ' '.join(tokenized_text)
    return combined_text

def clean_code(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.replace('[', ' ').replace(']', ' ').replace('(', ' ')\
    .replace(')', ' ').replace('{', ' ').replace('}', ' ').replace('=', ' ').replace(',', ' ')
    text = text.lower()
    text = text.replace('_', ' ')
    text = text.replace('\n', ' ')
    text = text.replace('.', ' ')
    text = re.sub(r'".*"', ' ', text)
    text = re.sub(r"'.*'", ' ', text)
    text = re.sub("^\d+\s|\s\d+\s|\s\d+$", ' ', text)
    text = re.sub(' +', ' ', text)
    text = text.strip()
    return text

def code_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_code(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    combined_text = ' '.join(tokenized_text)
    return combined_text

def count_hastags(row):
    "Count the number of hashtags "
    row['hash_count'] = row['source'].count('# ') if row['cell_type']=='markdown' else 0
    return row

## reading utils

In [6]:
def get_ranks(base, derived):
    return [base.index(d) for d in derived]

def read_train_data(data_dir, NUM_TRAIN = 10000, OFFSET=0):
    def read_notebook(path):
        return (
            pd.read_json(
                path,
                dtype={'cell_type': 'category', 'source': 'str'})
            .assign(id=path.stem)  # final path component
            .rename_axis('cell_id')
        )

    paths_train = list((data_dir / 'train').glob('*.json'))[OFFSET:NUM_TRAIN]
    notebooks_train = [
      read_notebook(path) for path in tqdm(paths_train, desc='Train NBs')
    ]
    df = (
      pd.concat(notebooks_train)
      .set_index('id', append=True)
      .swaplevel()
      .sort_index(level='id', sort_remaining=False)
    )
    return df

def get_df_orders_and_ranks(df, data_dir):
    # train orders
    df_orders = pd.read_csv(
      data_dir / 'train_orders.csv',
      index_col='id',
      squeeze=True,
    ).str.split()  # cell_ids str -> list


    df_orders_ = df_orders.to_frame().join(
      # reset only one index out of many -> "cell_id"; make a list out of cells in train data
      df.reset_index('cell_id').groupby('id')['cell_id'].apply(list),
      how='right',
    )

    ranks = {}
    for id_, cell_order, cell_id in df_orders_.itertuples():
        ranks[id_] = {'cell_id': cell_id, 'rank': get_ranks(cell_order, cell_id)}

    df_ranks = (
      pd.DataFrame
      .from_dict(ranks, orient='index')
      .rename_axis('id')
      .apply(pd.Series.explode)
      .set_index('cell_id', append=True)
    )
    # now we have
    # id cell_id rank
    return df_orders, df_ranks


def get_ancestors(data_dir, ids):
    # Split, keeping notebooks with a common origin (ancestor_id) together
    df_ancestors = pd.read_csv(data_dir / 'train_ancestors.csv', index_col='id')
    return df_ancestors.loc[ids, 'ancestor_id']

## Reading

In [49]:
# example pipeline work
df = read_train_data(data_dir, NUM_TRAIN=1000)
df_orders, df_ranks = get_df_orders_and_ranks(df, data_dir)

# Feature Engineering

In [8]:
from sklearn.preprocessing import FunctionTransformer

In [28]:
def prepare_data(df, func, new_column, on_column='source'):
    df[new_column] = df.progress_apply(lambda x: func(x[on_column]), axis=1)
    return df

In [50]:
#splitting on markdown and code
markdowns = df[df['cell_type'] == 'markdown']
codes = df[df['cell_type'] == 'code']

In [57]:
## lemmatizer 
lem = nltk.stem.wordnet.WordNetLemmatizer()

def tokenize_python_code_names(text):
    try:
        code_text = tokenize.generate_tokens(io.StringIO(text).readline)
        strings = [tok.string for tok in code_text if tok.type==1]
    except Exception:
        return None # Error happened
    return ' '.join(strings)

def tokenize_python_code_comments(text):
    try:
        code_text = tokenize.generate_tokens(io.StringIO(text).readline)
        strings = [tok.string for tok in code_text if tok.type==55]
    except Exception:
        return None # Error happened
    return ' '.join(strings)

def lemm_sentence(text):
    lst_txt = [lem.lemmatize(word) for word in text.split()]
    return ' '.join(lst_txt)

def collect_entities(text):
    "Named Entity Recognition"
    rs = ner(text)
    labels = []
    for r in rs.ents:
        labels.append(r.label_)
    return ' '.join(labels)
# Some example of results on 1000 ds
# [['CARDINAL' '851']
#  ['DATE' '561']
#  ['EVENT' '6']
# ...

## Code pipeline

In [69]:
code_pipeline = Pipeline([
    ('Clearing and parsing source', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': code_preprocessing,
             'new_column': "source_clean",  # TODO maybe even remove it or leave the same. check on gridsearch later
             'on_column': "source"})),
    
    ('Tokenizing python code names', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': tokenize_python_code_names,
             'new_column': "python_code_names",
             'on_column': "source"})),
    
    ('[1] Tokenizing python code comments', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': tokenize_python_code_comments,
             'new_column': "code_comments",
             'on_column': "source"})),
    ('[2] Lemmatizing code comments', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': tokenize_python_code_comments,
             'new_column': "code_comments",
             'on_column': "code_comments"})),
    ('[3] Clearing and parsing code comments', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': text_preprocessing, # todo maybe make additional pipeline for comments and multisteps pipe
             'new_column': "code_comments",
             'on_column': "code_comments"})),
    ('[4] Lemmatizing comments', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': lemm_sentence,
             'new_column': "code_comments_lemm",
             'on_column': "code_comments"})),
    # todo make comments statistics and implement it if column is not empty
    # ('[3] Comments stats', FunctionTransformer(
    #      func=prepare_data,
    #      kw_args={
    #          'func': lambda x: len(str(x).split()),
    #          'new_column': "len_of_code_comments",
    #          'on_column': "code_comments"})),
], verbose=True)
result = code_pipeline.fit_transform(codes)


[Pipeline]  (step 1 of 6) Processing Clearing and parsing source, total=   4.6s



[Pipeline]  (step 2 of 6) Processing Tokenizing python code names, total=  11.7s



[Pipeline]  (step 3 of 6) Processing [1] Tokenizing python code comments, total=  11.6s



[Pipeline]  (step 4 of 6) Processing [2] Lemmatizing code comments, total=   3.0s



[Pipeline]  (step 5 of 6) Processing [3] Clearing and parsing code comments, total=   3.8s



[Pipeline]  (step 6 of 6) Processing [4] Lemmatizing comments, total=   5.6s


In [74]:
import contextualSpellCheck

In [75]:
nlp = spacy.load('en_core_web_sm')
contextualSpellCheck.add_to_pipe(nlp)


660 ms ± 7.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%timeit nlp('There is no comin to consiousnes without pain')._.outcome_spellCheck

691 ms ± 8.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
result[result['code_comments']!='']

cell_type                                                                                                                   source  \
id             cell_id                                                                                                                                       
000e671b3324e2 17857a56      code  # construct train input\n\noutput = pd.DataFrame()\ngb = train.groupby('Patient')\ntk0 = tqdm(gb, total=len(gb))\nfo...   
               fba07254      code  # construct test input\n\ntest = pd.read_csv('../input/osic-pulmonary-fibrosis-progression/test.csv')\\n        .ren...   
               ffb99051      code  folds = train[[ID, 'Patient', TARGET]].copy()\n#Fold = KFold(n_splits=N_FOLD, shuffle=True, random_state=SEED)\nFold...   
               2afe8a24      code  #===========================================================\n# model\n#============================================...   
               d349aeae      code  target = train[TARGET]\ntest[TARGET] = np.nan\n\n# features\ncat_features = ['Sex', 'SmokingStatus']\nnum_features =...   
...                           ...                                                                                                                      ...   
ffccaad7a9a626 85134a3b      code                                                                                    # Dimensions of te Data\n\niris.shape   
               dae3e97d      code                                                               # Statistical summary of all attributes\n\niris.describe()   
               c698528a      code                          # To know how many variety of species are present in the data\n\niris.groupby('Species').size()   
               4d5ac628      code            # Selecting Dependent and Independent Variables\nX = iris.iloc[:, [1, 3]].values\ny = iris.iloc[:, -1].values   
               9b479da1      code  # Splitting into training and test sets\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_tr...   

                                                                                                                                    source_clean  \
id             cell_id                                                                                                                             
000e671b3324e2 17857a56            construct train input output pd dataframe gb train groupby reset index drop true print train shape train head   
               fba07254                                                         construct test input test pd read csv print test shape test head   
               ffb99051                                                                                     folds train id astype int folds head   
               2afe8a24  model from sklearn preprocessing import minmaxscaler scaler minmaxscaler def run single ridge param train df test df...   
               d349aeae  target train target test target np nan features cat features seed oof predictions run kfold ridge ridge param train ...   
...                                                                                                                                          ...   
ffccaad7a9a626 85134a3b                                                                                         dimensions of te data iris shape   
               dae3e97d                                                                      statistical summary of all attributes iris describe   
               c698528a                                            to know how many variety of species are present in the data iris groupby size   
               4d5ac628                                    selecting dependent and independent variables x iris iloc values y iris iloc 1 values   
               9b479da1  splitting into training and test sets from sklearn model selection import train test split x train x test y train y ...   

                                            

## markdowns pipeline

In [ ]:
code_pipeline = Pipeline([
    ('Clearing and parsing source', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': code_preprocessing,
             'new_column': "source_clean",
             'on_column': "source"})),
    ('Lemmatizing sentences', FunctionTransformer(
         func=prepare_data,
         kw_args={
             'func': code_preprocessing,
             'new_column': "source_clean",
             'on_column': "source"})),
    

], verbose=True)
result = code_pipeline.fit_transform(codes)

In [39]:
df = pd.concat([codes, markdowns])

In [40]:
# counting features
df['text_len'] = df['source_clean'].astype(str).apply(len)
df['text_word_count'] = df['source_clean'].apply(lambda x: len(str(x).split()))

# TODO make the right order

## Machine Learning glossary feature

In [42]:
import torch
from sentence_transformers import SentenceTransformer, util

In [43]:
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [44]:
ml_glossary = pd.read_csv("machine_learning_glossary_terms.csv")

In [45]:
# sentences we will be searching through
corpus = np.array(ml_glossary['definition'])
terms = np.array(ml_glossary['term'])
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

### implementing feature

In [46]:
# better do not vector but words and the put them to TFID
top_k=7
def get_top_glossary_terms(query):
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)
    hits = ['_'.join(str(terms[hit['corpus_id']]).split()) for hit in hits[0]]
    return ' '.join(hits)

In [47]:
df.loc[:, 'glossary_ml_terms'] = (
    df['source_clean'].apply(str).progress_apply(lambda x: get_top_glossary_terms(x)).copy().values
)

In [50]:
df['pct_rank'] = df.groupby(["id", "cell_type"])["rank"].apply(lambda s: pd.Series((np.arange(len(s)) + 1) /(len(s) + 1), index=s.index))

KeyError: 'Column not found: rank'

In [20]:
def mark_each_cell_with_its_position(current_X, full_df):
    """
    marking each cell with its number if its code, for markdown zero. 
    We are doing it to help the model learn the correct order in lines?
    """
    old_shape = current_X.shape
    current_X = sparse.hstack((
        current_X,
        np.where(
            full_df['cell_type'] == 'code',
            full_df.groupby(['id', 'cell_type']).cumcount().to_numpy() + 1,
            0,
        ).reshape(-1, 1)
    ))
    new_shape = current_X.shape
    print(f"cell with position change {old_shape} -> {new_shape}")
    return current_X

# idf(t) = log [ n / df(t) ] + 1, where df(t) – number of time term is used
tfidf = TfidfVectorizer(min_df=0.01, max_features=169)
def convert_to_TfidfVector(df):
    print("Converting with Tfid vectorizer")
    return tfidf.fit_transform(df.astype(str))

def add_data_to_sparse(current_X, values):
    print(f"Added {values.shape} to the dataframe")
    return sparse.hstack((
        current_X,
        values
    ))